In [1]:
import pandas as pd
from bokeh.models import ColumnDataSource, Select, CustomJS, HoverTool
from bokeh.plotting import figure, show
from bokeh.io import output_file
from bokeh.layouts import column, row

output_file("MLB_stats.html")

# Read batting average data
ba_dodgers = pd.read_csv('./Data/ba/dodgers.csv', usecols=["player", "team_abbrev", "ba"])
ba_guardians = pd.read_csv('./Data/ba/guardians.csv', usecols=["player", "team_abbrev", "ba"])
ba_mets = pd.read_csv('./Data/ba/mets.csv', usecols=["player", "team_abbrev", "ba"])
ba_yankees = pd.read_csv('./Data/ba/yankees.csv', usecols=["player", "team_abbrev", "ba"])

# Combine batting average data
ba = pd.concat([ba_dodgers, ba_guardians, ba_mets, ba_yankees], ignore_index=True)
ba = ba.sort_values(by=["ba"], ascending=False)
ba_dict = ba.to_dict(orient="list")

# Read home run data
hr_dodgers = pd.read_csv('./Data/homerun/dodgers.csv', usecols=["player", "team_abbrev", "hr_total"])
hr_guardians = pd.read_csv('./Data/homerun/guardians.csv', usecols=["player", "team_abbrev", "hr_total"])
hr_mets = pd.read_csv('./Data/homerun/mets.csv', usecols=["player", "team_abbrev", "hr_total"])
hr_yankees = pd.read_csv('./Data/homerun/yankees.csv', usecols=["player", "team_abbrev", "hr_total"])

# Combine home run data
hr = pd.concat([hr_dodgers, hr_guardians, hr_mets, hr_yankees], ignore_index=True)
hr = hr.sort_values(by=["hr_total"], ascending=False)
hr_dict = hr.to_dict(orient="list")

# Create data sources
source = ColumnDataSource(data=hr_dict)
source1 = ColumnDataSource(data=ba_dict)

# Create batting average figure
p_ba = figure(title="MLB Hitters Batting Average", x_axis_label="Players", y_axis_label="Batting Average",
               x_range=list(source1.data["player"]), width=2000, height=1200, tools="pan,wheel_zoom,box_zoom,reset",
               background_fill_color="#f2f2f2")
p_ba.vbar(x="player", top="ba", source=source1, legend_label="Batting Average", color="blue", width=0.9)
hover_ba = HoverTool()
hover_ba.tooltips = [("Player", "@player"), ("Team", "@team_abbrev"), ("Batting Average", "@ba")]
p_ba.add_tools(hover_ba)
p_ba.xaxis.major_label_orientation = 1

# Create home runs figure
p_hr = figure(title="MLB Hitters Home Runs", x_axis_label="Players", y_axis_label="Home Runs",
               x_range=list(source.data["player"]), width=2000, height=1200, tools="pan,wheel_zoom,box_zoom,reset", 
               background_fill_color="#f2f2f2")
p_hr.vbar(x="player", top="hr_total", source=source, legend_label="Home Runs", color="red", width=0.9)

hover_hr = HoverTool()
hover_hr.tooltips = [("Player", "@player"), ("Team", "@team_abbrev"), ("Home Runs", "@hr_total")]
p_hr.add_tools(hover_hr)
p_hr.xaxis.major_label_orientation = 1  # Rotate labels


team_select = Select(title="Team:", value="All", options=["All"] + list(hr["team_abbrev"].unique()))

sort_select = Select(title="Metric:", value="Home Runs", options=["Home Runs", "Batting Average"])

team_select.js_on_change("value", CustomJS(args=dict(source=source, source1=source1, hr=hr_dict, ba=ba_dict, p_hr=p_hr, p_ba=p_ba), code="""
    const selectedTeam = cb_obj.value;

    // Initialize filtered data for home runs
    const filteredDataHR = { player: [], team_abbrev: [], hr_total: [] };
    
    // Initialize filtered data for batting average
    const filteredDataBA = { player: [], team_abbrev: [], ba: [] };

    // Filter home runs data
    for (let i = 0; i < hr['player'].length; i++) {
        if (selectedTeam === "All" || hr['team_abbrev'][i] === selectedTeam) {
            filteredDataHR['player'].push(hr['player'][i]);
            filteredDataHR['team_abbrev'].push(hr['team_abbrev'][i]);
            filteredDataHR['hr_total'].push(hr['hr_total'][i]);
        }
    }

    // Filter batting average data
    for (let i = 0; i < ba['player'].length; i++) {
        if (selectedTeam === "All" || ba['team_abbrev'][i] === selectedTeam) {
            filteredDataBA['player'].push(ba['player'][i]);
            filteredDataBA['team_abbrev'].push(ba['team_abbrev'][i]);
            filteredDataBA['ba'].push(ba['ba'][i]);
        }
    }

    // Update data sources
    source.data = filteredDataHR;
    source1.data = filteredDataBA;

    // Update x_range for both figures
    p_hr.x_range.factors = filteredDataHR['player'];
    p_ba.x_range.factors = filteredDataBA['player'];
    
    // Emit changes
    source.change.emit();
    source1.change.emit();
"""))

sort_select.js_on_change("value", CustomJS(args=dict(p_hr=p_hr, p_ba=p_ba), code="""
    if (cb_obj.value === "Home Runs") {
        p_hr.visible = true;
        p_ba.visible = false;
    } else {
        p_hr.visible = false;
        p_ba.visible = true;
    }
"""))

# Display
layout = column(row(team_select, sort_select), p_ba, p_hr)
p_ba.visible = False  # Start with batting average hidden
show(layout)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
